In [27]:
import numpy as np
from PIL import ImageGrab
from PIL import Image
import cv2
import io
import time
%matplotlib inline 
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (15, 9)
import seaborn as sns

In [28]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
game_url = "game/dino.html"
chrome_driver_path = "../chromedriver.exe"
class Game:
    def __init__(self,custom_config=True):
        chrome_options = Options()
        chrome_options.add_argument("disable-infobars")
        self._driver = webdriver.Chrome(executable_path = chrome_driver_path,chrome_options=chrome_options)
        self._driver.set_window_position(x=-10,y=0)
        self._driver.set_window_size(200, 300)
        self._driver.get(os.path.abspath(game_url))
        if custom_config:
            self._driver.execute_script("Runner.config.ACCELERATION=0")
    def get_crashed(self):
        return self._driver.execute_script("return Runner.instance_.crashed")
    def get_playing(self):
        return self._driver.execute_script("return Runner.instance_.playing")
    def restart(self):
        return self._driver.execute_script("Runner.instance_.restart()")
    def press_up(self):
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ARROW_UP)
    def press_down(self):
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ARROW_DOWN)
    def get_score(self):
        score_array = self._driver.execute_script("return Runner.instance_.distanceMeter.digits")
        score = ''.join(score_array)
        return int(score)
    def pause(self):
        return self._driver.execute_script("return Runner.instance_.stop()")
    def resume(self):
        return self._driver.execute_script("return Runner.instance_.play()")
#     def grab_screen(self):
#         print("grab start")le
#         image = self._driver.get_screenshot_as_png()
# #         performance_measure.send(time.time())
#         image = np.fromstring(image, np.uint8)
# #         performance_measure.send(time.time())
#         image = cv2.imdecode(image,cv2.IMREAD_UNCHANGED) 
# #         performance_measure.send(time.time())
#         image = process_img(image)
# #         performance_measure.send(time.time())
# #         cv2.imwrite('screenshot.png',image)
#         return image
    def end(self):
        self._driver.close()

In [29]:
class DinoAgent:
    def __init__(self,game):
        self._game = game;
        self.jump();
    def is_running(self):
        return self._game.get_playing()
    def is_crashed(self):
        return self._game.get_crashed()
    def jump(self):
        self._game.press_up()
#         time.sleep(0.25)
    def duck(self):
        self._game.press_down()

In [30]:
#processing image as required
def process_img(image):
    #game is already in grey scale canvas, canny to get only edges and reduce unwanted objects(clouds)
#     image = cv2.Canny(image, threshold1 = 100, threshold2 = 200)
#     image = image[10:140,0:200] #img[y:y+h, x:x+w]
#     image = resized_image = cv2.resize(image, (80, 80)) 
    image = cv2.resize(image, (0,0), fx = 0.15, fy = 0.10)
    image = image[2:38,10:50] #img[y:y+h, x:x+w]
    image = cv2.Canny(image, threshold1 = 200, threshold2 = 200)
    return  image

In [31]:
def grab_screen(_driver = None):
    screen =  np.array(ImageGrab.grab(bbox=(0,180,400,400)))
    image = process_img(screen)
    if _driver!=None:
        image = _driver.get_screenshot_as_png()
    return image

In [32]:
# print(grab_screen().shape)
# game = Game()
# dino = DinoAgent(game)
# last_time = time.time()
# while(True):
    
# # #     print('loop took {} seconds'.format(time.time()-last_time))
# # #     last_time = time.time()
# # #     cv2.imwrite("./img_data/dino"+str(time())+".jpg",image)
# # #     dino.duck()
# #     #exit on q pres
# # #     print('{0} {1} '.format(r_t,end_t))
# # #     cv2.imshow('window',game.grab_screen())
    
#     image = grab_screen()
#     cv2.imshow('window',image)
#     print(image.shape[0])
# #     cv2.destroyAllWindows()

# # #     from matplotlib import pyplot as plt
# # #     plt.imshow(image)
# # #     plt.title('my picture')
# # #     plt.show()

# # #     grab_screen()
# #     if(dino.is_crashed()):
# #         #jumping starts the game again if dino has crashed
# # #         temp = (game.get_score())
# #         game.restart()
#     if (cv2.waitKey(25) & 0xFF == ord('q')):
#         cv2.destroyAllWindows()
#         game.end()
# #         cv2.imwrite('dino.jpg',image)
#         break

In [33]:
#game parameters
ACTIONS = 2 # possible actions: jump, do nothing
GAMMA = 0.5 # decay rate of past observations original 0.99
OBSERVATION = 1000. # timesteps to observe before training
EXPLORE = 30000 #300000. # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 20000 # number of previous transitions to remember
BATCH = 32 # size of minibatch
FRAME_PER_ACTION = 4
LEARNING_RATE = 1e-4

In [34]:
img_rows , img_cols = 40,20
img_channels = 4 #We stack 4 frames

In [35]:
def buildmodel():
    print("Now we build the model")
    model = Sequential()
    model.add(Convolution2D(32, 8, 8, subsample=(4, 4), border_mode='same',input_shape=(img_cols,img_rows,img_channels)))  #80*80*4
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 4, 4, subsample=(2, 2), border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode='same'))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(ACTIONS))
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam)
    print("We finish building the model")
    return model

In [36]:
def write_log(callback, names, logs, batch_no):
    for name, value in zip(names, logs):
        summary = tf.Summary()
        summary_value = summary.value.add()
        summary_value.simple_value = value
        summary_value.tag = name
        callback.writer.add_summary(summary, batch_no)
        callback.writer.flush()

In [37]:
def trainNetwork(model,game_state,observe=False):
    # open up a game state to communicate with emulator
    last_time = time.time()
    # store the previous observations in replay memory
    D = deque()
#     display = show_img()
#     display.__next__()
    # get the first state by doing nothing and preprocess the image to 80x80x4
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] =1 
    x_t, r_0, terminal = game_state.get_state(do_nothing)
    

    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)
    

    #In Keras, need to reshape
    s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*80*80*4


    if observe :#args['mode'] == 'Run':
        OBSERVE = 999999999    #We keep observe, never train
        epsilon = FINAL_EPSILON
        print ("Now we load weight")
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")    
    else:                       #We go to training mode
        OBSERVE = OBSERVATION
        epsilon = INITIAL_EPSILON
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)

    t = 0
    while (True):
        loss = 0
        Q_sa = 0
        action_index = 0
        r_t = 0
        a_t = np.zeros([ACTIONS])
        #choose an action epsilon greedy
        if t % FRAME_PER_ACTION == 0:
            if  random.random() <= epsilon:
                print("----------Random Action----------")
                action_index = random.randrange(ACTIONS)
                a_t[action_index] = 1
            else:
                q = model.predict(s_t)       #input a stack of 4 images, get the prediction
                max_Q = np.argmax(q)
                action_index = max_Q
                a_t[action_index] = 1
        #We reduced the epsilon gradually
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE #update to original asap

        #run the selected action and observed next state and reward
        x_t1, r_t, terminal = game_state.get_state(a_t)
        print('loop took {} seconds'.format(time.time()-last_time))
        last_time = time.time()
#       display.send(x_t1)
        x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1) #1x80x80x1
        s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3)

        # store the transition in D
        D.append((s_t, action_index, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        #only train if done observing
        if t > OBSERVE: 
            
            #sample a minibatch to train on
            minibatch = random.sample(D, BATCH)
            inputs = np.zeros((BATCH, s_t.shape[1], s_t.shape[2], s_t.shape[3]))   #32, 80, 80, 4
            targets = np.zeros((inputs.shape[0], ACTIONS))                         #32, 2

            #Now we do the experience replay
            for i in range(0, len(minibatch)):
                state_t = minibatch[i][0]
                action_t = minibatch[i][1]   #This is action index
                reward_t = minibatch[i][2]
                state_t1 = minibatch[i][3]
                terminal = minibatch[i][4]
                # if terminated, only equals reward

                inputs[i:i + 1] = state_t    #I saved down s_t

                targets[i] = model.predict(state_t)  # Hitting each buttom probability
                Q_sa = model.predict(state_t1)

                if terminal:
                    targets[i, action_t] = reward_t
                else:
                    targets[i, action_t] = reward_t + GAMMA * np.max(Q_sa)

            # targets2 = normalize(targets)
            loss += model.train_on_batch(inputs, targets)
            loss_df.loc[len(loss_df)] = loss
        else:
            time.sleep(0.16)
        s_t = s_t1
        t = t + 1
        
        # save progress every 10000 iterations
        if t % 1000 == 0:
            print("Now we save model")
            model.save_weights("model.h5", overwrite=True)
            with open("model.json", "w") as outfile:
                json.dump(model.to_json(), outfile)

        # print info
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"

        print("TIMESTEP", t, "/ STATE", state, \
            "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", r_t, \
            "/ Q_MAX " , np.max(Q_sa), "/ Loss ", loss)

    print("Episode finished!")
    print("************************")

In [38]:
import pandas as pd
import numpy as np
import _thread as thread
loss_df = pd.DataFrame(0, index=range(2),columns=['loss'])
scores_df = pd.DataFrame(0, index=range(2),columns=['score'])
actions_df = pd.DataFrame(0, index=range(2),columns=['action'])

In [39]:
def playGame(observe=False):
    game = Game()
    dino = DinoAgent(game)
    game_state = Game_sate(dino,game)
    model = buildmodel()
    trainNetwork(model,game_state,observe=observe)
    

In [40]:
class Game_sate:
    def __init__(self,agent,game):
        self._agent = agent
        self._game = game
        self._display = show_img()
        self._display.__next__()
    def get_state(self,actions):
        actions_df.loc[len(actions_df)] = actions[1]
        reward = 0.1
        is_over = False
        if actions[1] == 1:
            self._agent.jump()

        image = grab_screen()
        self._display.send(image)

        if self._agent.is_crashed():
            scores_df.loc[len(loss_df)] = self._game.get_score()
            self._game.restart()
            reward = -5
            is_over = True
        return image, reward, is_over

In [41]:
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD , Adam
import tensorflow as tf
from keras.callbacks import TensorBoard
from collections import deque
import random
import json

In [42]:
def show_img(graphs = False):
    """
    Show images in new window
    """
    frame = 0
    while True:
        screen = (yield)
        window_title = "logs" if graphs else "game_play"
        cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)        # Create window with freedom of dimensions
        imS = cv2.resize(screen, (800, 400)) 
        cv2.imshow(window_title, imS)
#         cv2.imwrite("screenshot"+str(frame)+".png",screen)
        if (cv2.waitKey(1) & 0xFF == ord('q')):
            cv2.destroyAllWindows()
            break

        frame += 1

In [43]:
playGame(observe=False)

Now we build the model
We finish building the model


C:\Users\ravi7\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (8, 8), input_shape=(20, 40, 4..., strides=(4, 4), padding="same")`
  after removing the cwd from sys.path.
C:\Users\ravi7\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (4, 4), strides=(2, 2), padding="same")`
  
C:\Users\ravi7\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), strides=(1, 1), padding="same")`
  


loop took 0.528921365737915 seconds
TIMESTEP 1 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.2015235424041748 seconds
TIMESTEP 2 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.21889281272888184 seconds
TIMESTEP 3 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.20416712760925293 seconds
TIMESTEP 4 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
----------Random Action----------
loop took 0.23114848136901855 seconds
TIMESTEP 5 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.20686578750610352 seconds
TIMESTEP 6 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.2173473834991455 seconds
TIMESTEP 7 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.2178940773010254 seconds
TIMESTEP 8 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 

TIMESTEP 67 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.23516321182250977 seconds
TIMESTEP 68 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.2497730255126953 seconds
TIMESTEP 69 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.21782326698303223 seconds
TIMESTEP 70 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.2191925048828125 seconds
TIMESTEP 71 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.2182924747467041 seconds
TIMESTEP 72 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.24936485290527344 seconds
TIMESTEP 73 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.22009873390197754 seconds
TIMESTEP 74 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.20492315292358398 seconds
TIM

loop took 0.23319244384765625 seconds
TIMESTEP 133 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.2042698860168457 seconds
TIMESTEP 134 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.21798229217529297 seconds
TIMESTEP 135 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.22484207153320312 seconds
TIMESTEP 136 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD -5 / Q_MAX  0 / Loss  0
loop took 0.1974947452545166 seconds
TIMESTEP 137 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.21708965301513672 seconds
TIMESTEP 138 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.2033395767211914 seconds
TIMESTEP 139 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
loop took 0.22130298614501953 seconds
TIMESTEP 140 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss

...

TIMESTEP 1394 / STATE explore / EPSILON 0.09869131000000164 / ACTION 0 / REWARD 0.1 / Q_MAX  0.0899048 / Loss  0.593283832073
loop took 0.17631864547729492 seconds
TIMESTEP 1395 / STATE explore / EPSILON 0.09868798000000165 / ACTION 0 / REWARD 0.1 / Q_MAX  -0.51634 / Loss  0.805286169052
loop took 0.19651103019714355 seconds
TIMESTEP 1396 / STATE explore / EPSILON 0.09868465000000165 / ACTION 0 / REWARD 0.1 / Q_MAX  0.196959 / Loss  1.9811412096
loop took 0.20754027366638184 seconds
TIMESTEP 1397 / STATE explore / EPSILON 0.09868132000000165 / ACTION 1 / REWARD 0.1 / Q_MAX  -0.477685 / Loss  0.775254011154
loop took 0.20341229438781738 seconds
TIMESTEP 1398 / STATE explore / EPSILON 0.09867799000000166 / ACTION 0 / REWARD 0.1 / Q_MAX  0.0302527 / Loss  1.70806241035
loop took 0.18552160263061523 seconds
TIMESTEP 1399 / STATE explore / EPSILON 0.09867466000000166 / ACTION 0 / REWARD 0.1 / Q_MAX  -0.145645 / Loss  0.461912512779
loop took 0.2225027084350586 seconds
TIMESTEP 1400 / STATE 

loop took 0.18726515769958496 seconds
TIMESTEP 1445 / STATE explore / EPSILON 0.09852148000000185 / ACTION 0 / REWARD 0.1 / Q_MAX  0.0481777 / Loss  0.720978736877
loop took 0.18725347518920898 seconds
TIMESTEP 1446 / STATE explore / EPSILON 0.09851815000000186 / ACTION 0 / REWARD 0.1 / Q_MAX  -0.242328 / Loss  0.87198984623
loop took 0.1694493293762207 seconds
TIMESTEP 1447 / STATE explore / EPSILON 0.09851482000000186 / ACTION 0 / REWARD 0.1 / Q_MAX  -0.48376 / Loss  0.582257628441
loop took 0.1761949062347412 seconds
TIMESTEP 1448 / STATE explore / EPSILON 0.09851149000000187 / ACTION 0 / REWARD 0.1 / Q_MAX  -0.7884 / Loss  0.304948508739
loop took 0.19859623908996582 seconds
TIMESTEP 1449 / STATE explore / EPSILON 0.09850816000000187 / ACTION 1 / REWARD 0.1 / Q_MAX  -0.281362 / Loss  1.19298815727
loop took 0.1714801788330078 seconds
TIMESTEP 1450 / STATE explore / EPSILON 0.09850483000000188 / ACTION 0 / REWARD 0.1 / Q_MAX  -0.0261157 / Loss  1.13314199448
loop took 0.191745519638

TIMESTEP 1496 / STATE explore / EPSILON 0.09835165000000207 / ACTION 0 / REWARD -5 / Q_MAX  0.0567913 / Loss  0.856238365173
loop took 0.22234463691711426 seconds
TIMESTEP 1497 / STATE explore / EPSILON 0.09834832000000207 / ACTION 0 / REWARD 0.1 / Q_MAX  -4.55814 / Loss  0.667400240898
loop took 0.20333456993103027 seconds
TIMESTEP 1498 / STATE explore / EPSILON 0.09834499000000207 / ACTION 0 / REWARD 0.1 / Q_MAX  -0.256724 / Loss  0.884163975716
loop took 0.19154644012451172 seconds
TIMESTEP 1499 / STATE explore / EPSILON 0.09834166000000208 / ACTION 0 / REWARD 0.1 / Q_MAX  0.0230429 / Loss  0.411145865917
loop took 0.1825113296508789 seconds
TIMESTEP 1500 / STATE explore / EPSILON 0.09833833000000208 / ACTION 0 / REWARD 0.1 / Q_MAX  0.0153309 / Loss  0.509860754013
loop took 0.19177699089050293 seconds
TIMESTEP 1501 / STATE explore / EPSILON 0.09833500000000209 / ACTION 1 / REWARD 0.1 / Q_MAX  -0.85703 / Loss  0.225203543901
loop took 0.2165687084197998 seconds
TIMESTEP 1502 / STATE

TIMESTEP 1546 / STATE explore / EPSILON 0.09818515000000227 / ACTION 0 / REWARD 0.1 / Q_MAX  -0.403539 / Loss  1.13104116917


StopIteration: 

In [ ]:


def show_plots():
    fig, axs = plt.subplots(ncols=2,nrows =2)
    loss_df['loss'] = loss_df['loss'].astype('float') 
    loss_df.plot(use_index=True,ax=axs[0,0])
    scores_df.plot(ax=axs[0,1])
    sns.distplot(actions_df,ax=axs[1,0])
    fig.canvas.draw()
    graph_img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    graph_img = graph_img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    disp = show_img(graphs=True)
    disp.__next__()
#     while True:
#         disp.send(graph_img)

In [ ]:
show_plots()